In [1]:
import numpy as np
from math import sqrt
import ot
import torch
from aux_functions import a_idx, w_indices, list_pairs, fast_w_indices, gen_4mom_approx, joint_wass_dist, \
    chen_error_3step, gen_2mom_approx, make_pretty

## Demonstration of the random variable which matches the first 4 moments of Levy Area

There is also a simple version which just matches the variances conditional on the W increment

In [2]:
a_true = np.genfromtxt('samples/samples_4-dim.csv', dtype=float, delimiter=',')[:(2*16384),4:10]

# generated4mom = gen_4mom_approx(4, bsz, _W=W)
# generated2mom = gen_2mom_approx(4, bsz, _W = W)
# print(generated2mom.shape)
print(a_true.shape)
# a_generated4mom = generated4mom[:, 4:10]
# a_generated2mom = generated2mom[:, 4:10]

(32768, 6)


In [4]:
err = [sqrt(ot.wasserstein_1d(a_true[:,i], a_generated4mom[:, i], p=2)) for i in range(4)]
print(err)

[0.0031215251014375177, 0.0031286148594010174, 0.002799245139591539, 0.0023474843729451474]


In [5]:
err = [sqrt(ot.wasserstein_1d(a_true[:,i], a_generated2mom[:, i], p=2)) for i in range(4)]
print(err)

[0.020255162715319474, 0.020892199738661764, 0.019990485264570586, 0.020879712144683007]


In [ ]:
joint_err = joint_wass_dist(a_true[:16384], a_true[16384:(2*16384)])
print(joint_err)

In [ ]:
joint_err = joint_wass_dist(a_true[:10000], a_generated2mom[:10000])
print(joint_err)

In [6]:
chen_error_3step(torch.tensor(generated4mom), 4)

/home/andy/anaconda3/envs/GAN/lib/python3.10/site-packages/ot/lp/solver_1d.py:41: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484746364/work/aten/src/ATen/native/TensorShape.cpp:2981.)
  cws = cws.T.contiguous()


[0.004951666707495244,
 0.004260804939288557,
 0.006073591261431599,
 0.004256758201574434,
 0.0037463570063839013,
 0.004579091179078815]

In [7]:
chen_error_3step(torch.tensor(generated2mom), 4)

[0.0203326034907954,
 0.020349840209122966,
 0.0192789607661177,
 0.020413672606116158,
 0.021174692502185056,
 0.021143816707986113]

In [4]:
for exp in range(3,21):
    bsz = 2**exp
    print(f"bsz: {bsz}  ch errors: {make_pretty(chen_error_3step(torch.tensor(samples[:bsz]), 4))}")

bsz: 8  ch errors: ['1.5791', '1.7491', '0.7755', '0.2971', '0.6300', '0.9556']
bsz: 16  ch errors: ['0.6855', '0.8728', '0.6793', '0.3425', '0.3331', '0.5892']
bsz: 32  ch errors: ['0.2926', '0.5134', '0.3787', '0.3781', '0.7878', '0.5238']
bsz: 64  ch errors: ['0.3111', '0.4807', '0.2955', '0.2368', '0.3462', '0.4252']
bsz: 128  ch errors: ['0.2190', '0.3330', '0.2821', '0.2071', '0.2395', '0.3477']
bsz: 256  ch errors: ['0.1521', '0.2389', '0.2814', '0.0901', '0.1506', '0.1531']
bsz: 512  ch errors: ['0.1504', '0.1037', '0.1145', '0.1093', '0.1016', '0.1999']
bsz: 1024  ch errors: ['0.1395', '0.1289', '0.1017', '0.0803', '0.0967', '0.1026']
bsz: 2048  ch errors: ['0.0616', '0.0476', '0.0785', '0.0848', '0.0689', '0.0780']
bsz: 4096  ch errors: ['0.0677', '0.0724', '0.0422', '0.0606', '0.0637', '0.0478']
bsz: 8192  ch errors: ['0.0402', '0.0527', '0.0383', '0.0301', '0.0374', '0.0522']
bsz: 16384  ch errors: ['0.0328', '0.0249', '0.0382', '0.0275', '0.0465', '0.0313']
bsz: 32768  ch 

In [ ]:
for exp in range(3,21):
    bsz = 2**exp
    print(f"bsz: {bsz}  ch errors: {make_pretty(chen_error_3step(torch.tensor(samples[:bsz]), 4))}")

## A utility for calculating the empirical fourth moments of a set of samples

In [8]:
def four_combos(n: int):
    lst = []
    for i in range(n):
        for j in range(i,n):
            for k in range(j,n):
                for l in range(k,n):
                    lst.append((i,j,k,l))
    return lst

def fourth_moments(input_samples: np.ndarray):
    dim = input_samples.shape[1]
    lst = four_combos(dim)
    res = []
    for i,j,k,l in lst:
        col = input_samples[:, i] * input_samples[:, j] * input_samples[:, k] * input_samples[:, l]
        res.append(col.mean())
    return res


w2 = np.concatenate((W,W,W,W), axis=0)
print(w2.shape)
combo_list = four_combos(3)
moms = fourth_moments(generated4mom)
moms2 = fourth_moments(a_true)
for i in range(len(combo_list)):
    print(f"combo: {combo_list[i]}, 4_match_RV moment: {moms[i] :.7f}, samples moment: {moms2[i] :.7f}")

[(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 0, 2), (0, 0, 1, 1), (0, 0, 1, 2), (0, 0, 2, 2), (0, 1, 1, 1), (0, 1, 1, 2), (0, 1, 2, 2), (0, 2, 2, 2), (1, 1, 1, 1), (1, 1, 1, 2), (1, 1, 2, 2), (1, 2, 2, 2), (2, 2, 2, 2)]


In [10]:
w2 = np.concatenate((W,W,W,W), axis=0)
print(w2.shape)

(262144, 3)


In [11]:
generated4mom = gen_4mom_approx(3, 4 * bsz, _W=w2)
moms = fourth_moments(generated4mom)
for i in range(len(combo_list)):
    print(f"combo: {combo_list[i]}, 4_match_RV moment: {moms[i] :.7f}, samples moment: {moms2[i] :.7f}")

combo: (0, 0, 0, 0), 4_match_RV moment: 0.1350575, samples moment: 0.1357276
combo: (0, 0, 0, 1), 4_match_RV moment: 0.0336348, samples moment: 0.0357896
combo: (0, 0, 0, 2), 4_match_RV moment: 0.0655758, samples moment: 0.0649001
combo: (0, 0, 1, 1), 4_match_RV moment: 0.0722153, samples moment: 0.0728633
combo: (0, 0, 1, 2), 4_match_RV moment: 0.0007437, samples moment: 0.0013768
combo: (0, 0, 2, 2), 4_match_RV moment: 0.0717766, samples moment: 0.0717662
combo: (0, 1, 1, 1), 4_match_RV moment: 0.0481259, samples moment: 0.0485544
combo: (0, 1, 1, 2), 4_match_RV moment: 0.0280197, samples moment: 0.0279942
combo: (0, 1, 2, 2), 4_match_RV moment: 0.0045032, samples moment: 0.0053913
combo: (0, 2, 2, 2), 4_match_RV moment: 0.0753795, samples moment: 0.0750950
combo: (1, 1, 1, 1), 4_match_RV moment: 0.2946441, samples moment: 0.2966960
combo: (1, 1, 1, 2), 4_match_RV moment: -0.0395968, samples moment: -0.0387191
combo: (1, 1, 2, 2), 4_match_RV moment: 0.0827522, samples moment: 0.08177